In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

In [4]:
op= Optimus(verbose=True)

Just check that Spark and all necessary environments vars are present...
-----
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
HADOOP_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
You don't have PYSPARK_PYTHON set
You don't have PYSPARK_DRIVER_PYTHON set
JAVA_HOME=C:\Program Files\Java\jdk1.8.0_181
Pyarrow Installed
-----
Starting or getting SparkSession and SparkContext...

                             ____        __  _                     
                            / __ \____  / /_(_)___ ___  __  _______
                           / / / / __ \/ __/ / __ `__ \/ / / / ___/
                          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
                          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
                              /_/                                  
                              
Transform and Roll out...
Optimus successfully imported. Have fun :).


In [5]:
from optimus.server import Server
s = Server()

In [ ]:
s.start()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [10]:
# https://stackoverflow.com/questions/89228/calling-an-external-command-in-python/92395#92395
import subprocess
return_code = subprocess.call("echo Hello World", shell=True) 

In [12]:
print(return_code)

0


In [ ]:
df = op.load.url("https://raw.githubusercontent.com/ironmussa/Optimus/master/examples/foo.csv")

In [1]:
terminal

NameError: name 'terminal' is not defined

In [ ]:
!pip install flask=0.12.2

In [12]:
df.table()

id 1 (int),firstName 2 (string),lastName 3 (string),billingId 4 (int),product 5 (string),price 6 (int),birth 7 (string),dummyCol 8 (string)
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down
